In [28]:
import numpy as np
import pandas as pd
import glob
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os 

 # ENTER WEEK YOU ARE CLEANING BELOW:

In [29]:
LIT_DATA_WEEK = 'Week 24'

In [30]:
warnings.filterwarnings('ignore')

# Paths for the Excel files
LIT_Outbound = r"W:\Logistics\Elliott\Callum\LIT\Outbound Looker"
Financial_Calendar = r"W:\Logistics\Elliott\Callum\LIT\Financial Calendar - 20-25.xlsx"
LIT_RTN = r"W:\Logistics\Elliott\Callum\LIT\Returns Looker"
OB_Status_Lookup = pd.read_excel(r"W:\Logistics\Elliott\Callum\LIT\Status Lookups.xlsx", sheet_name = 'OB Status Lookup')
RTN_Status_Lookup = pd.read_excel(r"W:\Logistics\Elliott\Callum\LIT\Status Lookups.xlsx", sheet_name = 'RTN Status Lookup')
Master_DATA = r"W:\Logistics\Elliott\Callum\LIT\Refund and Exchange Data"
for filename in os.listdir(Master_DATA):
    if filename.endswith('.xlsx'):
        file_path1 = os.path.join(Master_DATA, filename)
        
        # Read the CSV file into a DataFrame
        df_Master = pd.read_excel(file_path1, sheet_name = LIT_DATA_WEEK)

# Future Location for the Template
output_dir = r"W:\Logistics\Elliott\Callum\LIT\LIT Output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Current date for the filename
current_date = datetime.now().strftime("%Y-%m-%d")


# Filename for the Excel output
output_filename = f"LIT {LIT_DATA_WEEK} Outbound & Returns Data {current_date}.xlsx"
output_filepath = os.path.join(output_dir, output_filename)

# Step 1: Script for Obtaining Order Numbers to put into Looker

In [31]:
# selecting required row from CS report
Rows_Selected=['LIT: No Courier Movements','LIT: Damaged/Confirmed Lost','Returns Issue: No Courier Movements','Returns Issue: Unprocessed Order at DC','Returns Issue: Missing item(s)','Returns Issue: Damaged/Confirmed Lost']
OB_rows = ['LIT: No Courier Movements','LIT: Damaged/Confirmed Lost']
RTN_rows = ['Returns Issue: No Courier Movements','Returns Issue: Unprocessed Order at DC','Returns Issue: Missing item(s)','Returns Issue: Damaged/Confirmed Lost']
df_Master = df_Master[df_Master['Reason Description'].isin(Rows_Selected)]

output_dir2 = r"W:\Logistics\Elliott\Callum\LIT\Order Numbers"
if not os.path.exists(output_dir2):
    os.makedirs(output_dir2)
# Filename for the Excel output
output_filename2 = f"LIT {LIT_DATA_WEEK} Order Numbers {current_date}.xlsx"
output_filepath2 = os.path.join(output_dir2, output_filename2)   

In [32]:
# Join df_Master to FC
excel_df = pd.read_excel(Financial_Calendar)
selected_columns=['Date','Period','Week','FY']
excel_df=excel_df[selected_columns]
excel_df['Date'] = pd.to_datetime(excel_df['Date'],dayfirst=True,errors='coerce')
excel_df['Period']=excel_df['Period'].astype(object)
excel_df['Week']=excel_df['Week'].astype(object)
excel_df['FY']=excel_df['FY'].astype(object)

In [33]:
# Join by financial Calendar
df_Master= df_Master.merge(excel_df,
                                      left_on='Date',
                                      right_on='Date',
                                      how='left')

In [34]:
# Order Number Column for OB and RTN
selected_column = 'Original Order Number'
df_OB_OrderID=df_Master[df_Master['Reason Description'].isin(OB_rows)]
df_RTN_OrderID=df_Master[df_Master['Reason Description'].isin(RTN_rows)]
df_OB_OrderID = df_OB_OrderID[selected_column]
df_RTN_OrderID=df_RTN_OrderID[selected_column]

In [35]:
with pd.ExcelWriter(output_filepath2) as writer:
    df_OB_OrderID.to_excel(writer, sheet_name='OB Order Numbers', index=False)
    df_RTN_OrderID.to_excel(writer, sheet_name='RTN Order Numbers', index=False)
    
# THIS WILL PRINT EXCEL SHEET WITH THE ORDER NUMBERS YOU NEED

# Outbound Data Cleaning

In [36]:
# First, work on cleaning outbound data.

# Iterate over all files in the directory
for filename in os.listdir(LIT_Outbound):
    if filename.endswith('.csv'):
        file_path = os.path.join(LIT_Outbound, filename)
        
        # Read the CSV file into a DataFrame
        df_OB = pd.read_csv(file_path)
        
        
        # Join Normalized Status Sheet:
        df_OB = df_OB.merge(OB_Status_Lookup,
                                        left_on='Latest Carrier Status',
                                        right_on='Status',
                                        how='left')
        
        # Convert the 'Date' column to datetime for non-empty date rows
        df_OB['Consignor Status Update - Date'] = pd.to_datetime(df_OB['Consignor Status Update - Date'],dayfirst=True)
        
        # Sort the DataFrame by 'Date' in descending order for non-empty date rows
        df_OB = df_OB.sort_values(by='Consignor Status Update - Date', ascending=False)
        
       # Create a helper column for sorting that puts 'delivered' last - we want to see status's for duplicates that are non delivered
        df_OB['SortPriority'] = df_OB['Status Normalized'].apply(lambda x: 1 if x == 'Delivered' else 0)
        
        # Sort by the helper column first, then by 'Status' if needed - this sorts by date and put delivered status's at the bottom of the dataframe.
        df_OB.sort_values(by=['SortPriority', 'Consignor Status Update - Date'], ascending=[True, False],inplace=True)
        
        # Remove duplicates by 'OrderNumber', keeping the first occurrence - drops the second occurance
        df_OB = df_OB.drop_duplicates(subset='Order Number', keep='first')
        
        # Optionally, drop the helper column if no longer needed
        df_OB.drop(columns=['SortPriority'], inplace=True)
        
# Sort data by most recent date so lookup is dependant on most recent update
df_OB_cleaned = df_OB.sort_values(by='Consignor Status Update - Date',ascending=False)
df_OB_cleaned['Work Stream']='Outbound'
df_OB_cleaned=df_OB_cleaned.drop(columns=['Status','Consignor Status Update - Date'])
df_OB_cleaned.rename(columns={'Status Normalized':'Latest Status','Carrier Name':'Carrier','Latest Carrier Status': 'Status Non-Normalized','Order Number':'Original Order Number'},inplace=True)

# Returns Data Cleaning

In [37]:
# Work On Clearing Duplicates (keep most recent requested date if there are).
for filename in os.listdir(LIT_RTN):
    if filename.endswith('.csv'):
        file_path2 = os.path.join(LIT_RTN, filename)
        
        # Read the CSV file into a DataFrame
        df_RTN = pd.read_csv(file_path2)
        
        
        df_RTN = df_RTN.merge(RTN_Status_Lookup,
                                        left_on='Last Tracking Status',
                                        right_on='Status',
                                        how='left')
        
        # Convert the 'Date' column to datetime for non-empty date rows
        df_RTN['Requested Date'] = pd.to_datetime(df_RTN['Requested Date'],dayfirst=True)
        
        # Sort the DataFrame by 'Date' in descending order for non-empty date rows
        df_RTN = df_RTN.sort_values(by='Requested Date', ascending=False)
        
        
        # Create a helper column for sorting that puts 'delivered' last
        df_RTN['SortPriority'] = df_RTN['Status Normalized'].apply(lambda x: 1 if x == 'Delivered' else 0)
        
        # Sort by the helper column first, then by 'Status' if needed
        df_RTN.sort_values(by=['SortPriority', 'Requested Date'], ascending=[True, False],inplace=True)
        
        # Remove duplicates by 'OrderNumber', keeping the first occurrence
        df_RTN = df_RTN.drop_duplicates(subset='Order Number', keep='first')
        
          # Optionally, drop the helper column if no longer needed
        df_RTN.drop(columns=['SortPriority'], inplace=True)
 
# Sort data by most recent date, additionally change column names to join to OB data
df_RTN_cleaned = df_RTN.sort_values(by='Requested Date',ascending=True)
df_RTN_cleaned['Work Stream']='Return'
df_RTN_cleaned=df_RTN_cleaned.drop(columns=['Requested Date','Status',])
df_RTN_cleaned.rename(columns={'Status Normalized':'Latest Status','Last Tracking Status':'Status Non-Normalized','Order Number':'Original Order Number'},inplace=True)

# Step 2: Join and Detect any Non-Normalized Status:

In [38]:
df_LIT_cleaned = pd.concat([df_OB_cleaned,df_RTN_cleaned],ignore_index=True)
# want to see non-normalized status's that need to be normalized
non_norm=df_LIT_cleaned[df_LIT_cleaned['Latest Status'].isna()]
print(non_norm[['Status Non-Normalized','Work Stream']])
non_norm = non_norm[['Status Non-Normalized','Work Stream']]
non_norm.to_excel(r"W:\Logistics\Elliott\Callum\LIT\Non Normalised Status\Non Normalized Status.xlsx")

Empty DataFrame
Columns: [Status Non-Normalized, Work Stream]
Index: []


In [39]:

df_LIT_Final = df_Master.merge(df_LIT_cleaned,
                                      left_on='Original Order Number',
                                      right_on='Original Order Number',
                                      how='left')
df_LIT_Final = df_LIT_Final.fillna('No Data')
df_LIT_Final['Carrier'] = df_LIT_Final['Carrier'].str.replace(r'\s*\(CS\)', '', regex=True)

In [40]:
with pd.ExcelWriter(output_filepath) as writer:
   df_LIT_Final.to_excel(writer, sheet_name='Lost In Transit - Cleaned ', index=False)